### Imports

In [4]:
from typing import Tuple, Optional, Deque, List

In [5]:
from collections import deque

In [6]:
from random import shuffle

In [7]:
from IPython.display import HTML, display

In [8]:
from enum import Enum

### Logic

In [10]:
class Suit(Enum):
    SPADES = ('♠', 'black')
    HEARTS = ('♥', 'red')
    CLUBS = ('♣', 'black')
    DIAMONDS = ('♦', 'red')

    def __init__(self, symbol, color):
        self.symbol = symbol
        self.color = color

class Rank(Enum):
    TWO = '2'
    THREE = '3'
    FdOUR = '4'
    FIVE = '5'
    SIX = '6'
    SEVEN = '7'
    EIGHT = '8'
    NINE = '9'
    TEN = '10'
    JACK = 'J'
    QUEEN = 'Q'
    KING = 'K'
    ACE = 'A'

class Card:
    def __init__(self, rank: Rank, suit: Suit):
        self.rank = rank
        self.suit = suit

    def __str__(self):
        return f"{self.rank.value}{self.suit.symbol}"

    def __eq__(self, other):
        return isinstance(other, Card) and self.rank == other.rank and self.suit == other.suit # Makes sure that other is a card object

    def __hash__(self):
        return hash((self.rank, self.suit))

class Deck:
    def __init__(self):
        self.cards = []
        self.burned = set()
        self.dealt = set()
        self._populate()

    def _populate(self):
        for suit in Suit:
                self.cards.append(Card(rank, suit))

    def deal(self, count=1):
        dealt = self.cards[:count] # Syntax has to do with slicing: Grabs first count cards from deck
        self.dealt.update(dealt) # Adds cards to self.dealt set (Update is extend() for sets)
        self.cards = self.cards[count:] # Removes first count cards from self.cards
        return dealt

    def burn(self):
        if self.cards:
            burn_card = self.cards.pop(0)
            self.burned.add(burn_card)
            return burn_card

    def shuffle(self):
        """Shuffles deck in place, in the future, will want a function that returns a copy for simulations."""
        # Moves sets back into the deck, and clears the sets, another way to do it would be to use the extend funciton
        # For the addition to a list
        self.cards += list(self.dealt) + list(self.burned)
        self.dealt.clear()
        self.burned.clear()
        shuffle(deck.cards)

In [11]:
POSITIONS_BY_PLAYER_COUNT = {
    2: ["Button", "Big Blind"],
    3: ["Button", "Small Blind", "Big Blind"],
    4: ["Button", "Small Blind", "Big Blind", "UTG"],
    5: ["Button", "Small Blind", "Big Blind", "UTG", "CO"],
    6: ["Button", "Small Blind", "Big Blind", "UTG", "HJ", "CO"],
    7: ["Button", "Small Blind", "Big Blind", "UTG", "MP", "HJ", "CO"],
    8: ["Button", "Small Blind", "Big Blind", "UTG", "UTG+1", "MP", "HJ", "CO"],
    9: ["Button", "Small Blind", "Big Blind", "UTG", "UTG+1", "UTG+2", "MP", "HJ", "CO"],
    10: ["Button", "Small Blind", "Big Blind", "UTG", "UTG+1", "UTG+2", "MP", "LJ", "HJ", "CO"]
}

In [54]:
class Player:
    def __init__(self, 
                 name: str, 
                 stack: float, 
                 is_bot: bool = False, 
                 seat_index: int = -1, # Unassigned until seated 
                 in_hand: bool = True, 
                 has_folded: bool = False, 
                 current_bet: float = 0.0, 
                 hole_cards: Optional[Tuple[Card, Card]] = None
            ):
        
        self.name = name
        self.stack = stack
        self.is_bot = is_bot
        self.seat_index = seat_index
        self.in_hand = in_hand
        self.current_bet = current_bet
        self.hole_cards = hole_cards
        self.position: Optional[str] = None

    def reset_for_new_hand(self):
        """Resets player attributes for a new hand. Doen't affect in_hand attribute, becuase They may be dealt out,
           or they are spectating or something."""
        self.hole_cards = None
        self.has_folded = False
        self.current_bet = 0
        
        

class Game:
    def __init__(self,
                 small_blind: float = 0.25,
                 big_blind: float = 0.50,
                 players: Optional[List[Player]] = None
            ):
        
        self.players: Deque[Player] = deque(players) if players else deque()
        self.community_cards = []
        self.dealer_index = 0
        self.pot = 0.0
        self.phase = "predeal"
        self.action_log = []
        self.deck = Deck()
        self.small_blind = small_blind
        self.big_blind = big_blind
        self.minimum_raise = big_blind

    def add_player(self, player_or_list):
        """Adds list of players, or single player to the game."""
        if isinstance(player_or_list, list):
            for p in player_or_list:
                self._add_single_player(p)

        else:
            self._add_single_player(player_or_list)

    def _add_single_player(self):
        player.seat_index = len(self.players)
        self.players.append(player)

    def start_new_hand(self):
        self.rotate_dealer()
        self.reset_players()
        self.assign_player_positions()
        self.deck.shuffle()
        self.community_cards = []
        self.pot = 0.0
        self.phase = Phase.PREFLOP
        self.post_blinds()
        self.deal_hole_cards()
        self.run_preflop()

    def rotate_dealer(self):
        """Rotate The dealer"""
        n = len(self.players)
        for i in range(1, n + 1):
            next_index = (self.dealer_index + i) % n
            if self.players[next_index].in_hand:
                self.dealer_index = next_index

    def assign_player_positions(self):
        """Assign official position names (e.g. Button, SB, BB)
           to all active players based on dealer index."""
        active_players = [p for p in self.players if p.in_hand]
        num_active = len(active_players)

        position_helper = Position(num_active)

        seat_to_player = {player.seat_index: player for player in active_players}
        ordered_seats = []
        for i in range(num_active):
            seat = (self.dealer_index + i) % len(self.players)
            if seat in seat_to_player:
                ordered_seats.append(seat)

        for offset, seat_index in enumerate(ordered_seats):
            player = seat_to_player[seat_index]
            position_name = position_helper.positions[offset]
            player.position = position_name

    def post_blinds(self):
        """Deduct small and big blinds from the appropiate players and add to the pot.
           If either player has less then the appropiate blind, they are forced all in."""
        small_blind_player = None
        big_blind_player = None

        for player in self.players:
            if not player.in_hand:
                continue
            if player.position == "Small Blind":
                small_blind_player = player
            elif player.position == "Big Blind":
                blg_blind_player = player

        if small_blind_player is None or big_blind_player is None:
            raise ValueError("Both small and Big blind players must be set before posting blinds.")

        small_blind_amount = min(self.small_blind, small_blind.player.stack)
        small_blind_player.stack -= small_blind_amount
        small_blind_player.current_bet = small_blind_amount
        self.pot += small_blind_amount
        self.action_log(f{small_blind_player.name} post small blind of {})

class Position:
    def __init__(self, num_players: int):
        if not (2 <= num_players <= 10):
            raise ValueError("Only 2 to 10 players supported.")
        self.positions = POSITIONS_BY_PLAYER_COUNT[num_players]

    def get_position(self, seat_index: int, button_index: int) -> str:
        """Given a seat_index and the button's index, return the official position name."""
        offset = (seat_index - button_index) % len(self.positions)
        return self.positions[offset]

class Action(Enum):
    FOLD = 'fold'
    CALL = 'call'
    CHECK = 'check'
    BET = 'bet'
    RAISE = 'raise'

class Phase(Enum):
    WAITING = "waiting"
    PREDEAL = "predeal"
    PREFOLP = "preflop"
    FLOP = "flop"
    TURN = "turn"
    RIVER = "river"
    SHOWDOWN = "showdown"

### Printing Logic

In [14]:
def styled_card_html(card):
    """Return an HTML-formatted card string with styling for Jupyter."""
    base_style = ("display:inline-block; font-family:monospace; font-size:1.2em; "\
                 "background:white; color:black; padding: 2px 6px; border-radius: 4px;"
    )

    suit_color = "red" if card.suit.color == "red" else "black"
    return f'<span style="{base_style}">{card.rank.value}<span style="color:{suit_color}">{card.suit.symbol}</span></span>'

Display Function to group cards by suit

In [16]:
def styled_deck_html(deck):
    lines = []

    for suit in Suit:
        # Collect all cards of this suit
        suit_cards = [c for c in deck.cards + list(deck.dealt) + list(deck.burned) if c.suit == suit]

        # Sort by rank using Rank enum order
        suit_cards.sort(key=lambda c: list(Rank).index(c.rank))

        line = ''
        for card in suit_cards:
            # Color logic
            if card in deck.burned:
                bg = "lightcoral"
                text = "black"
            elif card in deck.dealt:
                bg = "lightblue"
                text = "black"
            else:
                bg = "white"
                text = "black"

            style = (
                f"background:{bg}; color:{text}; display:inline-block; "
                f"font-family:monospace; font-weight:bold; font-size:1.1em; "
                f"padding:2px 6px; border-radius:4px; margin:1px;"
            )

            suit_color = "red" if card.suit.color == "red" else text
            html = f'<span style="{style}">{card.rank.value}<span style="color:{suit_color}">{card.suit.symbol}</span></span>'
            line += html

        lines.append(f'<div style="margin-bottom:15px;">{line}</div>')

    return '<div style="line-height: 2;">' + ''.join(lines) + '</div>'


### Testing

In [18]:
second_card = Card(Rank.JACK, Suit.CLUBS)
card = Card(Rank.NINE, Suit.HEARTS)
display(HTML(styled_card_html(second_card)))
display(HTML(styled_card_html(card)))

In [19]:
deck = Deck()

In [20]:
deck.shuffle()

In [21]:
dealt_card = deck.deal()
burn_card = deck.burn()
dealt2 = deck.deal()
burn2 = deck.burn()

In [22]:
display(HTML(styled_deck_html(deck)))

### Ramdom & Depricated

In [24]:
# Adding doc string would be cool.
Card?

Init signature: Card(rank: __main__.Rank, suit: __main__.Suit)
Docstring:      <no docstring>
Type:           type
Subclasses:     

### Notes

<span style="color: green">5/3/2025</span>

Working on the Deck class currently. Have to implement hash and eq special methods to make it work in a set? Then i will need to create a display function to group cards by suit. uses lambda to sort. i am not all that familiar with that. I also imported display from IPython.display, which i did not have before

<span style="color: red">Dunder Eq and Hash</span>

dunder eq & hash come into play when doing some_set = {card1, card2} so python can tell if they are the same.

eq defines equality between objects.

hash enables objects to use in sets and dicts: Python requires objects in sets/dicts to have a <b>hash value</b>: Number that stays the same as long as the object's value doesnt change. (Unique value for sets and maps)

<span style="color: red">Lambda functions</span>

<span style="color: red">IPython.display</span>

<span style="color: green">5/3/2025</span>

Worked on getting the deck to display with burnt & dealt cards. Next want to work on a shuffle feature I think. It might be an easy change to get the deck to display with Ace's first. that is really just a small thing. Then I will eventually need to add players, gameflow, position, calculating, hands and pot odds. 

<span style="color: green">5/4/2025</span>

<span style="color: red">Git & GitHub</span>

<span style="color: red">Uploading</span>

Uploaded to Git & Github

- git clone https://github.com/your-username/your-repo.git
- cd your-repo
- mv /path/to/your_notebook.ipynb .
- git add your_notebook.ipynb
- git commit -m "Add my notebook"
- git push origin main

Now I have a local git repository in my /poker folder linked to GitHub.

If I were to move the entire folder (repository) everything works as long as I 

- navigate to new location to run Git commands
- Git only works inside repo directory (Cant push/ pull from outside of it.)

Once in Repo folder

- git add "my_notebook"
- git commit -m "Add some message"
- git push origin main

<span style="color: Red">Branching</span>

- git checkout -b new-feature
- make changes ....
- git add . (or file, cause I dont want to commit the database.)
- git commit -m "Message"
- git push origin main new-feature (push new branch to github, but for a solo project, terminal is faster.)

<span style="color: Red">Merging</span>

- git checkout main
- git pull origin main (make sure it's up to date.)
- git merge new-feature

If that Succeeds...

- git push origin main (Update github with merged result)
- git branch -d new-feature (delete local branch)
- git push origin --delete new-feature (delete remote branch)

Or use GitHub Interface...

- Push your branch: git push origin new-feature
- Go to GitHub, click "Compare & Pull Request"
- Review and click "Merge"
- Delete branch via button

<span style="color: Red">Stashing</span>

Like puting your changes in a drawer and coming back to them later...

- git stash save "WIP: notebook update" (stash with a message)
- git stash list (list al stashes)
- git stash pop stash@{1} (restore specific stash)
- git stash drop stash@{0} (discard stash if no longer need it

<span style="color: Red">Naming Convention</span>

_populate() or _funcition() means to say that: "This is an implementaiton detail, and you probably shouldn't use it directly."

<span style="color: green">5/5/2025</span>

<span style="color: red">Game, Players, Position....</span>

<span style="color: red">Modulo Arithmetic</span>

Finding the remainder when one number is divided by another. Often used for <b>Wraparound Logic</b> for example...

(11 + 2) % 12 (adding two hours at 11pm to a 12 hour clock.)


<span style="color: green">5/6/2025</span>

Make classes more robust with optional defaults.